## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

* если источник traffic_source равен yandex или google, то в source_type ставится organic
* для источников paid и email из России - ставим ad
* для источников paid и email не из России - ставим other
* все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')

def source_type(row):
    if row.traffic_source in ['yandex', 'google']:
        return 'organic'
    elif row.traffic_source in ['paid', 'email']:
        return 'ad' if row.region == 'Russia' else 'other'
    return row.traffic_source

log['source_type'] = log.apply(source_type, axis=1)

log.loc[10:20]

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
10,1549980777,01d448afd9,https://host.ru/a5dda93e70318570c0,India,27e76a4a80,direct,direct
11,1549980785,71b6c1d538,https://host.ru/c8d9213a31839f9a3a,Ukraine,3b13701488,google,organic
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,ad
13,1549980792,dd46c3eebb,https://host.ru/3004a8273caeef2867,China,7e301c8c41,direct,direct
14,1549980793,e7ecebbc59,https://host.ru/05e41944dd85a900a6,Belarus,ac52dedc3b,direct,direct
15,1549980797,5322fecfc9,https://host.ru/4adc8a29b212051b04,Belarus,9a45d98867,paid,other
16,1549980801,374708fff7,https://host.ru/39fa884393666d45fc,Russia,e9bd59e22d,google,organic
17,1549980807,0a88111852,https://host.ru/2fcb223266880ccf57,Ukraine,a2ba8fa3db,paid,other
18,1549980814,60daa3a5f7,https://host.ru/ef57543002ce2a0f2a,Belarus,72cb10fb5c,direct,direct
19,1549980816,d6fd62f5ce,https://host.ru/f1eb4601740d627ab0,Belarus,7269dcd869,direct,direct


## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

* Прочитайте содержимое файла с датафрейм
* Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [2]:
import pandas as pd

urls = pd.read_csv('URLs.txt')

urls[urls.url.str.contains(r'/\d{8}-')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')

lifetime = pd.pivot_table(
    ratings, 
    index='userId', 
    values=['rating', 'timestamp'], 
    aggfunc={'rating':'count', 'timestamp':lambda x: x.max() - x.min()}
)

lifetime[lifetime.rating >= 100].mean().timestamp

39331748.56653992

## Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [4]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

revenue = rzd.merge(auto, how='outer').merge(air, how='outer').merge(auto, how='outer', ).fillna(0)
revenue_with_address = revenue.merge(client_base)

revenue.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0


In [5]:
revenue_with_address.head()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
